<a href="https://colab.research.google.com/github/dipanjanS/mastering-intelligent-agents-langgraph-workshop-dhs2025/blob/main/Module-2-Building-Simple-Agentic-AI-Systems/M2LC2_Build_a_Tool_Use_ReAct_Agentic_AI_System_with_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Build a Tool-Use ReAct Agentic AI System with CrewAI

## Problem Statement

In this notebook, we will be building an Agentic AI system — HealthBuddy — using CrewAI with built-in utility functions. This system will act as an intelligent assistant that can:

- Understand user queries about health
- Research using web and arXiv tools
- Recommend appropriate doctors based on user symptoms

We will follow the **ReAct principle** (Reason + Act) to let the agent think, use tools when needed, observe results, and provide a well-formed final answer. Instead of relying on built-in LLM knowledge alone, the agent will use **external tools** to gather accurate and helpful information before responding.


---

### Objective:

Our goals in this notebook are:

- **Build a ReAct-based Tool-Use Agent using CrewAI**
- Equip the agent with multiple tools (web search, pubmed search, doctor recommendation)
- Handle user queries end-to-end — from interpreting intent to delivering a useful, cited, and grounded response
- Simulate a multi-step reasoning and tool-using workflow

By the end of this notebook, we will have a working agent that can research a health query, gather relevant information, and offer advice including suggesting a doctor - all through structured decision-making and tool usage.

### Agent Architecture:

The following figure shows the agent architecture including all the components and the overall workflow

![](https://i.imgur.com/i78Sybb.png)


In [ ]:
!pip install crewai[tools]==0.157.0 langchain==0.3.27 langchain-community==0.3.27 arxiv==2.2.0 pymupdf==1.26.3 --quiet

## Load Necessary Dependencies

In [ ]:
import json
from langchain_core.messages import SystemMessage, HumanMessage
from IPython.display import display, Image, Markdown

## Setup Authentication and LLM Client

Here we authenticate and connect to necessary LLM Clients using OpenAI Authentication

In [ ]:
import os
import getpass

# OpenAI API Key (for chat & embeddings)
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key (https://platform.openai.com/account/api-keys):\n")

# Tavily API Key (for web search)
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key (https://app.tavily.com/home):\n")

In [ ]:
# Disable CrewAI telemetry only
os.environ['CREWAI_DISABLE_TELEMETRY'] = 'true'

# Disable all OpenTelemetry (including CrewAI)
os.environ['OTEL_SDK_DISABLED'] = 'true'

In [ ]:
from crewai import LLM

llm_client = LLM(
    model="openai/gpt-4o-mini", # call model by provider/model_name
    temperature=0,
)

## Preparing Database for Doctor Recommendations Tool

In this section, we will create a small, in-memory database that contains information about doctors. This data will be used by our **Doctor Recommendation Tool**, which will help users find the right doctor based on their health query or symptoms.

The database includes a list of doctors along with their:
- Name
- Specialization (e.g., Dermatology, Pediatrics, Cardiology)
- Location
- Availability
- Contact information

We are using a simple Python list of dictionaries to store the doctor data. This keeps it easy to understand and modify. In a real-world application, this would typically be replaced by a backend database like PostgreSQL, MongoDB, or an external API.

We will build a tool later on to use this data to recommend doctors based on the user's needs — for example, suggesting a pediatrician for a child’s fever or a cardiologist for chest pain.

In [ ]:
# loading our doctors dataset
doctors_db = [
    {"name": "Dr. Janet Dyne", "specialization": "Endocrinology (Diabetes Care)", "available_timings": "10:00 AM - 1:00 PM", "location": "City Health Clinic", "contact": "janet.dyne@healthclinic.com"},
    {"name": "Dr. Don Blake ", "specialization": "Cardiology (Heart Specialist)", "available_timings": "2:00 PM - 5:00 PM", "location": "Metro Cardiac Center", "contact": "don.blake@metrocardiac.com"},
    {"name": "Dr. Susan D'Souza", "specialization": "Oncology (Cancer Care)", "available_timings": "11:00 AM - 2:00 PM", "location": "Hope Cancer Institute", "contact": "susan.dsouza@hopecancer.org"},
    {"name": "Dr. Matt Murdock", "specialization": "Psychiatry (Mental Health)", "available_timings": "4:00 PM - 7:00 PM", "location": "Mind Care Center", "contact": "matt.murdock@mindcare.com"},
    {"name": "Dr. Dinah Lance", "specialization": "General Physician", "available_timings": "9:00 AM - 12:00 PM", "location": "Downtown Medical Center", "contact": "dinah.lance@downtownmed.com"}
]

## Create Tools for AI Agent

In this section, we will define the tools that our AI Agent will use to perform specific tasks.

CrewAI makes it easy to create and register tools using the `tool` decorator. A tool includes:
- A name and description
- The python function to be called
- An input schema that tells the model what arguments it can use

When tools are defined properly, they help the model solve more complex problems by letting it take actions and use external data. This makes the system more useful and reliable.

### 🧪 Example
```python
from crewai.tools import tool

@tool("search_web")
def search_web(query: str) -> str:
    """Get live information for user queries from the web."""
    # assuming we have a google_search function implemented to search on google
    return google_search(query)
```

These tools will allow the agent to retrieve information from our preloaded vector databases (web search and PubMed), as well as recommend doctors from our in-memory doctor database.

The goal is to modularize the logic for different types of tasks into reusable components that can be invoked by the LLM when needed. These include:

- A **Web Search Tool** that queries the web to get relevant information through web search
- A **ArXiv Search Tool** that retrieves information from top research papers from arxiv.org relevant to the query  
- A **Doctor Recommendation Tool** that finds suitable doctors based on user symptoms or needs

This tool-based setup is essential for enabling agentic behavior, where the LLM reasons through a problem, decides which action to take, and requests to call the right tools to gather more information or perform a task.


In [ ]:
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.retrievers import ArxivRetriever
from crewai.tools import tool

# Tool for web search on general health topics
# Tavily Web Search
tavily_search = TavilySearchAPIWrapper()

@tool("search_web")
def search_web(query: str) -> list:
    """
    Search the web for general or up-to-date information on healthcare topics.

    Inputs:
    - query (str): The search query string. Should describe the healthcare topic or information you want to find.

    Outputs:
    - list: A list of up to 3 formatted strings, each containing:
        - Title of the search result
        - Content extracted from the page
        - Source URL
    """
    results = tavily_search.raw_results(query=query, max_results=3, search_depth='advanced',
                                        include_answer=False, include_raw_content=True)
    docs = results['results']
    docs = [doc for doc in docs if doc.get("raw_content") is not None]
    docs = ['## Title\n'+doc['title']+'\n\n'+'## Content\n'+doc['raw_content']+'\n\n'+'##Source\n'+doc['url'] for doc in docs]
    return docs


# Tool for arXiv Search
# arxiv search retriever
arxiv_retriever = ArxivRetriever(
    top_k_results=3,
    get_full_documents=True,
    doc_content_chars_max=20000
)

@tool("search_arxiv")
def search_arxiv(query: str) -> list:
    """
    Search arXiv for relevant scientific research papers and articles.

    Inputs:
    - query (str): The research topic or keywords to search for on arXiv.

    Outputs:
    - list: A list of up to 3 formatted strings, each containing:
        - Title of the paper
        - Summary of the paper
        - Full content (truncated to maximum allowed characters)
      Returns ["No articles found for the given query."] if no matches are found.
    """
    try:
        results = arxiv_retriever.invoke(query)
        if results:
            articles = ['## Title\n'+doc.metadata['Title']+'\n\n'+'## Summary\n'+doc.metadata['Summary']+'\n\n'+'##Content\n'+doc.page_content for doc in results]
            return articles
        else:
            return ["No articles found for the given query."]
    except Exception as e:
        return [f"Error fetching arXiv articles: {str(e)}"]


@tool("recommend_doctor")
def recommend_doctor(query: str) -> dict:
    """
    Recommend the most suitable doctor based on the user's symptoms or health-related query.

    Inputs:
    - query (str): A description of the patient's symptoms or healthcare needs.

    Outputs:
    - dict: A dictionary containing:
        - "recommended_doctor": JSON-formatted details of the selected doctor from the `doctors_db`.
          If the most suitable match cannot be determined, defaults to recommending the General Physician.
    """
    doctors_list = str(doctors_db)
    prompt = f"""You are an assistant helping recommend a doctor based on patient's health issues.

                 Here is the list of available doctors:
                {doctors_list}

                Given the user's query: "{query}"

                Choose the most suitable doctor from the list. Only pick one doctor.
                Return only the selected doctor's information in JSON format (no markdown).
                If not sure, recommend the General Physician.
              """

    response = llm_client.call(prompt)
    return {"recommended_doctor": response}


In [ ]:
recommend_doctor

## Define Tools to be Used by the Agent

In this section, we define the tools that our AI agent will use when reasoning through user queries. These include:

- `search_web`: To fetch general health-related information from the internet.
- `search_arxiv`: To retrieve scientific research papers from arXiv.org.
- `recommend_doctor`: To recommend a suitable doctor based on the user's symptoms.

These tools are already defined and registered using the `@tool` decorator. Here, we simply collect them into a list to be fed into the agent function later on.


In [ ]:
# List of all tools that the LLM should be aware of
# These tools were defined earlier using the @tool decorator
tools = [search_web, search_arxiv, recommend_doctor]

## Define Agent Instructions Prompt

To guide the LLM-based agent, we provide a custom system prompt that sets the role and behavior of the assistant.

The prompt clearly instructs the agent to:
- Act as a helpful healthcare assistant
- Reason on input queries and do the following:
  - Research the query using the most relevant tools (web, pubmed)
  - Recommend a doctor only if appropriate

This prompt plays a critical role in shaping how the agent reasons, decides when to call tools, and how to construct final responses. It supports ReAct-style behavior where the model reflects, takes actions (via tools), and continues reasoning.

In [ ]:
# Instruction prompt for the overall Agent
AGENT_SYS_PROMPT = r"""You are an agent designed to act as an expert in researching on medical symptoms
and also recommend relevant doctors for booking appointments. Also remember the current year is 2025.

Given an input user query call relevant tools and give the most appropriate response.
Follow some of these guidelines to help you make more informed decisions:
  - If the user's query specifies recommending a doctor only then recommend an appropriate doctor using the recommend_doctor tool
  - If the user is researching on detailed and specific aspects around symptoms, treatments and other aspects related to healthcare
  use both search_web and search_arxiv tools to get comprehensive information and then give a well-structured response
  - If the user is just looking for general information around healthcare then web search is enough
  - Use search_arxiv tool only if the query is related to information which might be found in research papers
  - Response should include cited source links and \ or arXiv Article Title, Publication Dates if available
  - If recommending doctors then only use the recommend_doctor tool and show detailed information in a nice structured way and recommend them to book an appointment via email
  - Politely decline answering any queries not related to medical or healthcare information
"""

## Create our Tool-Use AI Agent

Using the tools, prompt, and LLM setup, we now create the actual **Tool-Use Agent** using CrewAI's built-in `Agent` utility class.

This creates a simple ReAct-style agent capable of:
- Receiving a user query
- Reasoning on next steps
- Calling the appropriate tool(s) if needed
- Observing tool outputs
- Returning a final answer

This step brings together everything we’ve built so far: tools, instructions, and reasoning logic into a functioning agentic loop.

In [ ]:
from crewai import Agent

BACKSTORY_TEXT = AGENT_SYS_PROMPT  # keep the exact system prompt content
GOAL_TEXT = "A health assistant who helps people with giving detailed comprehensive answers on healthcare queries and recommends appropriate doctors"

healthbuddy_agent = Agent(
    role="HealthBuddy Assistant",
    goal=GOAL_TEXT,
    backstory=BACKSTORY_TEXT,
    tools=[search_web, search_arxiv, recommend_doctor],
    llm=llm_client
)

## View our Agent Flow



In [ ]:
healthbuddy_agent

## Build Utility Function to Stream Agent Results

We define a helper function to stream the step-by-step output of the agent. This makes it easier to trace:
- What the agent doing in each step
- Which tool it decides to call
- What response it gets from that tool
- How it forms the final reply

Streaming output is helpful when evaluating multi-step reasoning and debugging tool use in real time.

In [ ]:
from crewai import Task, Crew

AGENT_GUIDELINES = r"""
Useful guidelines:
  - If the user's query specifies recommending a doctor only then recommend an appropriate doctor using the recommend_doctor tool
  - If the user is researching on detailed and specific aspects around symptoms, treatments and other aspects related to healthcare
  use both search_web and search_arxiv tools to get comprehensive information and then give a well-structured response
  - If the user is just looking for general information around healthcare then web search is enough
  - Use search_arxiv tool only if the query is related to information which might be found in research papers
  - Response should include cited source links and \ or arXiv Article Title, Publication Dates if available
  - If recommending doctors then only use the recommend_doctor tool and show detailed information in a nice structured way and recommend them to book an appointment via email
"""

def run_crewai_query(agent, query, verbose=True):
    task = Task(
        description=f"{AGENT_GUIDELINES}\n\nUser query: {query}",
        expected_output="Comprehensive response based on user query",
        agent=agent,
        markdown=True
    )
    crew = Crew(agents=[agent],
                tasks=[task],
                verbose=verbose)
    return crew.kickoff()

## Test out our Agent!

In this final section, we run a complete test of our Tool-Use AI Agent by passing it a sample health-related query.

We observe how the agent:
- Interprets the query
- Decides which tool(s) to use (if any)
- Executes tool calls
- Streams the intermediate steps and final output

In [ ]:
query = "what are the latest methods for diabetes management and recommend a doctor please"
response = run_crewai_query(agent=healthbuddy_agent, query=query, verbose=True)

In [ ]:
display(Markdown(response.raw))

In [ ]:
# Example usage
query = "I am having panic attacks, what could I do? get detailed comprehensive information please"
response = run_crewai_query(agent=healthbuddy_agent, query=query, verbose=False)

In [ ]:
display(Markdown(response.raw))

In [ ]:
# Example usage without printing detailed log messages
query = "I am having panic attacks, please recommend a right doctor"
response = run_crewai_query(agent=healthbuddy_agent, query=query, verbose=True)

In [ ]:
display(Markdown(response.raw))